<a href="https://colab.research.google.com/github/Pebeirao/TensorFlow-Basic-Tutorial/blob/master/Save_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-beta1'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

W0712 10:30:16.454521 140518319871872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.2462 - accuracy: 0.6429
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 632us/sample - loss: 1.1891 - accuracy: 0.6580 - val_loss: 0.7184 - val_accuracy: 0.7740
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4348 - accuracy: 0.8823
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 346us/sample - loss: 0.4297 - accuracy: 0.8840 - val_loss: 0.5452 - val_accuracy: 0.8280
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2951 - accuracy: 0.9278
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 363us/sample - loss: 0.2902 - accuracy: 0.9280 - val_loss: 0.4808 - val_accuracy: 0.8440
Epoch 4/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.1953 - accuracy: 0.9531
Epoch

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 119us/sample - loss: 2.3082 - accuracy: 0.1170
Untrained model, accuracy: 11.70%


In [8]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 87us/sample - loss: 0.4149 - accuracy: 0.8710
Restored model, accuracy: 87.10%


In [9]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0712 11:24:45.103927 140518319871872 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 138us/sample - loss: 0.4940 - accuracy: 0.8760
Restored model, accuracy: 87.60%


In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 124us/sample - loss: 0.4940 - accuracy: 0.8760
Restored model, accuracy: 87.60%


In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

W0712 11:25:46.635889 140518319871872 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0712 11:25:46.637594 140518319871872 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0712 11:25:46.638710 140518319871872 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0712 11:25:46.639506 140518319871872 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0712 11:25:46.641394 140518319871872 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0712 11:25:46.642262 140518319871872 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 322us/sample - loss: 1.2113 - accuracy: 0.6590
Epoch 2/5
1000/1000 [==============================] - 0s 192us/sample - loss: 0.4394 - accuracy: 0.8700
Epoch 3/5
1000/1000 [==============================] - 0s 178us/sample - loss: 0.2937 - accuracy: 0.9180
Epoch 4/5
1000/1000 [==============================] - 0s 188us/sample - loss: 0.2141 - accuracy: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 185us/sample - loss: 0.1641 - accuracy: 0.9640


In [15]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 113us/sample - loss: 0.4203 - accuracy: 0.8510
Restored model, accuracy: 85.10%


In [17]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 284us/sample - loss: 1.1573 - accuracy: 0.6670
Epoch 2/5
1000/1000 [==============================] - 0s 181us/sample - loss: 0.4310 - accuracy: 0.8730
Epoch 3/5
1000/1000 [==============================] - 0s 191us/sample - loss: 0.2884 - accuracy: 0.9230
Epoch 4/5
1000/1000 [==============================] - 0s 190us/sample - loss: 0.2070 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 186us/sample - loss: 0.1587 - accuracy: 0.9650


In [18]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0712 11:26:25.866080 140518319871872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0712 11:26:25.867664 140518319871872 export_utils.py:182] Export includes no default signature!
W0712 11:26:26.124268 140518319871872 export_utils.py:182] Export includes no default signature!


'./saved_models/1562930785'

In [19]:
!ls saved_models/

1562930785


In [20]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.predict(test_images).shape

(1000, 10)

In [22]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 117us/sample - loss: 0.4244 - accuracy: 0.8600
Restored model, accuracy: 86.00%
